In [1]:
#Install the packages we need, including Google OR-Tools 
%%capture
!pip install ortools
!pip install pandas
!pip install numpy
!pip install matplotlib

!apt install gdal-bin python-gdal python3-gdal 
%pip install pandas fiona shapely pyproj
!apt install python3-rtree 
!pip install geopandas==0.10.0
!pip install descartes
!pip install scipy 
!pip install statsmodels
!pip install gdal
!pip install scikit-learn
!pip install statsmodels
!pip install networkx
!pip install osmnx

In [2]:
#Here we are importing the packages we need. 
import geopandas as gpd
import pandas as pd 
from geopandas.tools import sjoin
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import shape
from descartes import PolygonPatch
import time
import math
import scipy.stats as stats
import numpy as np
import os, sys
from pyproj import CRS, Transformer
import fiona

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import matplotlib as mpl
from math import floor

from shapely.ops import unary_union

import warnings
warnings.filterwarnings('ignore')

from osgeo import ogr, gdal,osr

from ortools.linear_solver import pywraplp
import string

import networkx as nx
import osmnx as ox

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

#Navigate to folder where data is stored in the drive. 
%cd /content/drive/MyDrive/jameslab/

dirname = '/content/drive/MyDrive/jameslab/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/jameslab


In [4]:
def get_potential_sites(gdf,spacing): 

    bounds = gdf.bounds
    xmax = bounds['maxx']
    xmin= bounds['minx']
    ymax = bounds['maxy']
    ymin = bounds['miny']

    num_col = int((xmax - xmin) / spacing) 
    num_row = int((ymax - ymin) / spacing)

    Yi = np.linspace(ymin,ymax,num_row+1)
    Xi = np.linspace(xmin,xmax,num_col+1)
    Xi,Yi = np.meshgrid(Xi,Yi)

    points = [Point(x) for x in zip(Xi.flatten(),Yi.flatten())]
    points = gpd.GeoDataFrame(geometry=gpd.GeoSeries(points),crs='esri:102001')\
    .to_crs('EPSG:4326')

    points['lat'] = list(points['geometry'].y)
    points['lon'] = list(points['geometry'].x)


    return points

In [5]:
# Create a regular grid of points in the area of interest, which is the bounding
#box of the 2020 (maximum) defoliation extent 

sa = gpd.read_file('study_area/area_of_interest.shp').to_crs('esri:102001')
sites_df = get_potential_sites(sa,500)

In [6]:
def join_shapefile(gdf,aerial_survey,col_name,fire=False,harvest=False,road=False): 
    if fire == False and harvest == False and road == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['FOREST_DAM'] = points_in_shp['FOREST_DAM'].fillna(0)
        points_in_shp.loc[points_in_shp['FOREST_DAM'] != 0, 'FOREST_DAM'] = 1
        points_in_shp[col_name] = points_in_shp['FOREST_DAM']

    elif harvest == True and fire == False and road == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['HarvestCon'] = points_in_shp['HarvestCon'].fillna(0)
        points_in_shp.loc[points_in_shp['HarvestCon'] != 0, 'HarvestCon'] = 1
        points_in_shp[col_name] = points_in_shp['HarvestCon']     

    elif road == True and fire == False and harvest == False: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['featurecla'] = points_in_shp['featurecla'].fillna(0)
        points_in_shp.loc[points_in_shp['featurecla'] != 0, 'featurecla'] = 1
        points_in_shp[col_name] = points_in_shp['featurecla'] 

    else: 
        points_in_shp = sjoin(gdf, aerial_survey, how='left')
        points_in_shp['FIRE_ID'] = points_in_shp['FIRE_ID'].fillna(0)
        points_in_shp.loc[points_in_shp['FIRE_ID'] != 0, 'FIRE_ID'] = 1
        points_in_shp[col_name] = points_in_shp['FIRE_ID']      

    array = np.array(points_in_shp[col_name])
    Xi = np.array(points_in_shp['lon'])
    Yi = np.array(points_in_shp['lat'])

    new_lon = []
    new_lat = []
    new_val = [] 
    for val,lon,lat in zip(array.flatten(),Xi.flatten(),Yi.flatten()):
        new_lon.append(lon)
        new_lat.append(lat)
        new_val.append(val)    

    points = [Point(x) for x in zip(Xi.flatten(),Yi.flatten())]

    #new_data = gpd.GeoDataFrame(geometry=gpd.GeoSeries(points),crs='EPSG:4326') 
    #new_data['lon'] = new_lon
    #new_data['lat'] = new_lat
    gdf[col_name] = new_val

    return gdf

In [7]:
def get_dist_boundary(points,polygons,year): 
    new_gdf = gpd.GeoDataFrame(points,geometry=points['geometry'])
    
    proj_p = polygons.to_crs('esri:102001')
    buff = proj_p.buffer(50)
    #buff = unary_union([mp for mp in buff])
    diff = proj_p.symmetric_difference(buff)
    
    diff = unary_union([mp for mp in diff])
    
    plc = pd.DataFrame() 
    plc['hold'] = [1]

    polygons = gpd.GeoDataFrame(plc,geometry=[diff])
    print(polygons)
    points = points.to_crs('esri:102001')
    points_in_shp = sjoin(points, polygons, how='left')
    print(points_in_shp)
    points_in_shp['in_boundary'+str(year)] = points_in_shp['hold'].fillna(0)
    print(points_in_shp)
    points_in_shp.loc[points_in_shp['in_boundary'+str(year)] != 0, 'in_boundary'+str(year)] = 1
    print(points_in_shp)
    list_insert = list(points_in_shp['in_boundary'+str(year)])
    
    new_gdf = new_gdf.loc[:,~new_gdf.columns.duplicated()]

    new_gdf['in_boundary'+str(year)] = list_insert
    print(new_gdf[new_gdf['in_boundary'+str(year)] < 1])
    return points_in_shp

In [8]:
fires = gpd.read_file('site_selection/fire_extract_2014_plus_correct.shp')
fire_df = join_shapefile(sites_df,fires,'fire_detected',fire=True)
print(fire_df[fire_df['fire_detected'] >= 1])

                           geometry        lat        lon  fire_detected
220245   POINT (-80.70997 45.84868)  45.848684 -80.709966              1
220246   POINT (-80.70376 45.84766)  45.847657 -80.703764              1
220247   POINT (-80.69756 45.84663)  45.846630 -80.697561              1
221400   POINT (-80.70853 45.85312)  45.853118 -80.708528              1
221401   POINT (-80.70233 45.85209)  45.852092 -80.702325              1
...                             ...        ...        ...            ...
1221655  POINT (-79.18732 49.63519)  49.635187 -79.187322              1
1221656  POINT (-79.18062 49.63407)  49.634072 -79.180622              1
1222811  POINT (-79.17891 49.63843)  49.638435 -79.178907              1
1222812  POINT (-79.17221 49.63732)  49.637320 -79.172206              1
1223967  POINT (-79.17049 49.64168)  49.641682 -79.170490              1

[2018 rows x 4 columns]


In [9]:
mortality7 = gpd.read_file('aerial_survey_maps/2014_ON_2022_02_10_CR.shp')
mortality7 = mortality7[mortality7['FOREST_DAM'] == 'Mortality']
ca7_df = join_shapefile(fire_df,mortality7,'ca7_detected',fire=False)

In [10]:
print(ca7_df)

mortality6 = gpd.read_file('aerial_survey_maps/2015_ON_2022_02_10_CR.shp')
mortality6 = mortality6[mortality6['FOREST_DAM'] == 'Mortality']
ca6_df = join_shapefile(ca7_df,mortality6,'ca6_detected',fire=False)

mortality5 = gpd.read_file('aerial_survey_maps/2016_ON_2022_02_10_CR.shp')
mortality5 = mortality5[mortality5['FOREST_DAM'] == 'Mortality']
ca5_df = join_shapefile(ca6_df,mortality5,'ca5_detected',fire=False)

mortality4 = gpd.read_file('aerial_survey_maps/2017_ON_2022_02_10_CR.shp')
mortality4 = mortality4[mortality4['FOREST_DAM'] == 'Mortality']
ca4_df = join_shapefile(ca5_df,mortality4,'ca4_detected',fire=False)

mortality3 = gpd.read_file('aerial_survey_maps/2018_ON_2022_02_10_CR.shp')
mortality3 = mortality3[mortality3['FOREST_DAM'] == 'Mortality']
ca3_df = join_shapefile(ca4_df,mortality3,'ca3_detected',fire=False)

mortality2 = gpd.read_file('aerial_survey_maps/2019_ON_2022_02_10_CR.shp')
mortality2 = mortality2[mortality2['FOREST_DAM'] == 'Mortality']
ca2_df = join_shapefile(ca3_df,mortality2,'ca2_detected',fire=False)

mortality1 = gpd.read_file('aerial_survey_maps/2020_ON_2022_02_10_CR.shp')
mortality1 = mortality1[mortality1['FOREST_DAM'] == 'Mortality']
ca1_df = join_shapefile(ca2_df,mortality1,'ca1_detected',fire=False)

                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1267030  POINT (-76.92042 49.41249)  49.412493 -76.920419              0   
1267031  POINT (-76.91381 49.41123)  49.411231 -76.913814              0   
1267032  POINT (-76.90721 49.40997)  49.409969 -76.907209              0   
1267033  POINT (-76.90060 49.40871)  49.408706 -76.900605              0   
1267034  POINT (-76.89400 49.40744)  49.407443 -76.894001              0   

         ca7_detected  
0                   0  
1                   0  
2              

In [11]:
ca7_df = ca1_df
mortality7 = gpd.read_file('aerial_survey_maps/2014_ON_2022_02_10_CR.shp')
mortality7 = mortality7[mortality7['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
  ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality7,2014)
print(list(ca7_df))
mortality6 = gpd.read_file('aerial_survey_maps/2015_ON_2022_02_10_CR.shp')
mortality6 = mortality6[mortality6['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality6,2015)
print(list(ca7_df))
mortality5 = gpd.read_file('aerial_survey_maps/2016_ON_2022_02_10_CR.shp')
mortality5 = mortality5[mortality5['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality5,2016)
print(list(ca7_df))
mortality4 = gpd.read_file('aerial_survey_maps/2017_ON_2022_02_10_CR.shp')
mortality4 = mortality4[mortality4['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality4,2017)
print(list(ca7_df))
mortality3 = gpd.read_file('aerial_survey_maps/2018_ON_2022_02_10_CR.shp')
mortality3 = mortality3[mortality3['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality3,2018)
print(list(ca7_df))
mortality2 = gpd.read_file('aerial_survey_maps/2019_ON_2022_02_10_CR.shp')
mortality2 = mortality2[mortality2['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality2,2019)
print(list(ca7_df))
mortality1 = gpd.read_file('aerial_survey_maps/2020_ON_2022_02_10_CR.shp')
mortality1 = mortality1[mortality1['FOREST_DAM'] == 'Mortality']
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
ca7_df = get_dist_boundary(ca7_df,mortality1,2020)
if 'index_right' in list(ca7_df): 
    ca7_df = ca7_df.drop('index_right', 1)
if 'hold' in list(ca7_df): 
  ca7_df = ca7_df.drop('hold', 1)
print(list(ca7_df))

   hold                                           geometry
0     1  MULTIPOLYGON (((1084157.207 729046.867, 108416...
                                geometry        lat        lon  fire_detected  \
0          POINT (792880.077 677729.051)  45.678324 -85.901291              0   
1          POINT (793380.099 677729.051)  45.677642 -85.895018              0   
2          POINT (793880.121 677729.051)  45.676960 -85.888745              0   
3          POINT (794380.143 677729.051)  45.676278 -85.882473              0   
4          POINT (794880.165 677729.051)  45.675594 -85.876200              0   
...                                  ...        ...        ...            ...   
1267030  POINT (1367905.302 1226168.220)  49.412493 -76.920419              0   
1267031  POINT (1368405.324 1226168.220)  49.411231 -76.913814              0   
1267032  POINT (1368905.346 1226168.220)  49.409969 -76.907209              0   
1267033  POINT (1369405.368 1226168.220)  49.408706 -76.900605          

In [12]:
def get_species(gdf,interpolated_surface,transform,\
                    size,srcds,col_name):
    '''This is a function to get the value inside the fire.
    We will use to calculate the mean, median, max value for a fire.
    
    Parameters
    ----------
        gdf : GeoPandas DataFrame
            gdf containing potential site points
        interpolated_surface : ndarray
            an array of values in the study area
        transform : list 
            list describing GeoTransform of raster 
        size : list 
            pixel dimensions
        srcds : GDAL object
            read in raster
            
    Returns
    ----------
        gdf 
            GeoPandas DataFrame with species values as column
    '''


    #Get information about the raster - what is pixel size? What is the origin?
    xOrigin = transform[0]
    yOrigin = transform[3]
    xMax = xOrigin + transform[1] * size[0]
    yMin = yOrigin + transform[5] * size[1]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    value = []
    lon = []
    lat = [] 
    
    # Iterate through the points within the fire 
    for idx,p in gdf.iterrows():
        #mx , my is the lon lat 
        mx,my=np.array(p['geometry'].coords.xy[0])[0], np.array(p['geometry'].coords.xy[1])[0]
        col = int((mx - xOrigin) / pixelWidth)
        row = int((yOrigin - my ) / pixelHeight) 
        # Calculate offset on the array 
        if row < interpolated_surface.shape[0] and col < interpolated_surface.shape[1]: 
            # Index the array where the point is and send to list to store it 
            value.append(interpolated_surface[row][col])
        else: 
            value.append(np.nan)

        lon.append(mx)
        lat.append(my)

    #Add values in list to the dataframe 
    gdf[col_name] = value

    return gdf

In [13]:
ca1_df = ca1_df.to_crs('EPSG:4326')
ca2_df = ca2_df.to_crs('EPSG:4326')
ca3_df = ca3_df.to_crs('EPSG:4326')
ca4_df = ca4_df.to_crs('EPSG:4326')
ca5_df = ca5_df.to_crs('EPSG:4326')
ca6_df = ca6_df.to_crs('EPSG:4326')
ca7_df = ca7_df.to_crs('EPSG:4326')

In [14]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_age.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_age = get_species(ca7_df,data,transform,(cols,rows,),src_ds,'age')
print(df_age)
#df_age = df_age.dropna(how='any')

                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1267030  POINT (-76.92042 49.41249)  49.412493 -76.920419              0   
1267031  POINT (-76.91381 49.41123)  49.411231 -76.913814              0   
1267032  POINT (-76.90721 49.40997)  49.409969 -76.907209              0   
1267033  POINT (-76.90060 49.40871)  49.408706 -76.900605              0   
1267034  POINT (-76.89400 49.40744)  49.407443 -76.894001              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
0     

In [15]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_bf.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_bf = get_species(df_age,data,transform,(cols,rows,),src_ds,'Bf')
print(df_bf)
#df_age = df_age.dropna(how='any')

                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1267030  POINT (-76.92042 49.41249)  49.412493 -76.920419              0   
1267031  POINT (-76.91381 49.41123)  49.411231 -76.913814              0   
1267032  POINT (-76.90721 49.40997)  49.409969 -76.907209              0   
1267033  POINT (-76.90060 49.40871)  49.408706 -76.900605              0   
1267034  POINT (-76.89400 49.40744)  49.407443 -76.894001              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
0     

In [16]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_sw.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_sw = get_species(df_bf,data,transform,(cols,rows,),src_ds,'Sw')
print(df_sw)

                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1267030  POINT (-76.92042 49.41249)  49.412493 -76.920419              0   
1267031  POINT (-76.91381 49.41123)  49.411231 -76.913814              0   
1267032  POINT (-76.90721 49.40997)  49.409969 -76.907209              0   
1267033  POINT (-76.90060 49.40871)  49.408706 -76.900605              0   
1267034  POINT (-76.89400 49.40744)  49.407443 -76.894001              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
0     

In [17]:
# Open selected raster using gdal 
src_ds = gdal.Open('site_selection/beaudoin_sb.tif')
# Get band 1 
rb1=src_ds.GetRasterBand(1)
transform=src_ds.GetGeoTransform()
cols = src_ds.RasterXSize
rows = src_ds.RasterYSize

# Convert raster to array 
data = rb1.ReadAsArray(0, 0, cols, rows)
df_sb = get_species(df_sw,data,transform,(cols,rows,),src_ds,'Sb')
print(df_sb)

                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1267030  POINT (-76.92042 49.41249)  49.412493 -76.920419              0   
1267031  POINT (-76.91381 49.41123)  49.411231 -76.913814              0   
1267032  POINT (-76.90721 49.40997)  49.409969 -76.907209              0   
1267033  POINT (-76.90060 49.40871)  49.408706 -76.900605              0   
1267034  POINT (-76.89400 49.40744)  49.407443 -76.894001              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
0     

In [18]:
#Get 7 dataframes for the 7 sites
df_sb = df_sb.dropna(how='any')
print(df_sb)
site1 = df_sb[df_sb['ca6_detected'] == 1]
print(site1)


                           geometry        lat        lon  fire_detected  \
0        POINT (-85.90129 45.67832)  45.678324 -85.901291              0   
1        POINT (-85.89502 45.67764)  45.677642 -85.895018              0   
2        POINT (-85.88875 45.67696)  45.676960 -85.888745              0   
3        POINT (-85.88247 45.67628)  45.676278 -85.882473              0   
4        POINT (-85.87620 45.67559)  45.675594 -85.876200              0   
...                             ...        ...        ...            ...   
1266794  POINT (-78.48960 49.69822)  49.698216 -78.489602              0   
1266795  POINT (-78.48291 49.69706)  49.697057 -78.482911              0   
1266796  POINT (-78.47622 49.69590)  49.695897 -78.476219              0   
1266797  POINT (-78.46953 49.69474)  49.694736 -78.469528              0   
1266798  POINT (-78.46284 49.69358)  49.693575 -78.462837              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
0     

In [19]:
harvest = gpd.read_file('site_selection/harvest_buffer.shp')

harvest_df = join_shapefile(df_sb,harvest,'harvest_detected',harvest=True)
print(harvest_df[harvest_df['harvest_detected'] >= 1])

                           geometry        lat        lon  fire_detected  \
279810   POINT (-83.75043 46.53814)  46.538141 -83.750429              0   
279811   POINT (-83.74408 46.53732)  46.537318 -83.744076              0   
280965   POINT (-83.74926 46.54261)  46.542608 -83.749256              0   
280966   POINT (-83.74290 46.54178)  46.541785 -83.742903              0   
282151   POINT (-83.55121 46.52135)  46.521352 -83.551211              0   
...                             ...        ...        ...            ...   
1201535  POINT (-82.49888 50.04871)  50.048713 -82.498883              0   
1202689  POINT (-82.50433 50.05402)  50.054018 -82.504331              0   
1202690  POINT (-82.49749 50.05312)  50.053121 -82.497488              0   
1203844  POINT (-82.50294 50.05843)  50.058426 -82.502936              0   
1203845  POINT (-82.49609 50.05753)  50.057528 -82.496093              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
279810

In [20]:
road = gpd.read_file('site_selection/road_buffer_1km.shp')

road_df = join_shapefile(harvest_df,road,'road_detected',road=True)
print(road_df[road_df['road_detected'] == 1])

                           geometry        lat        lon  fire_detected  \
7106     POINT (-84.79375 45.57881)  45.578811 -84.793752              0   
7107     POINT (-84.78751 45.57805)  45.578055 -84.787508              0   
7108     POINT (-84.78126 45.57730)  45.577298 -84.781265              0   
7109     POINT (-84.77502 45.57654)  45.576541 -84.775022              0   
8261     POINT (-84.79270 45.58330)  45.583303 -84.792700              0   
...                             ...        ...        ...            ...   
1204177  POINT (-80.24066 49.73488)  49.734882 -80.240658              0   
1204178  POINT (-80.23392 49.73384)  49.733836 -80.233916              0   
1205330  POINT (-80.25253 49.74135)  49.741351 -80.252531              0   
1205331  POINT (-80.24579 49.74031)  49.740306 -80.245788              0   
1205332  POINT (-80.23905 49.73926)  49.739261 -80.239045              0   

         ca7_detected  ca6_detected  ca5_detected  ca4_detected  ca3_detected  \
7106  

In [21]:
def get_dist_driving(G, gdf1,gdf2): 
    site_options = {} 
    for x,y in zip(gdf1['lon'],gdf1['lat']):
        df = pd.DataFrame() 
        dist_list = [] 
        lon_list = [] 
        lat_list = [] 
        for x2,y2 in zip(gdf2['lon'],gdf2['lat']): 
            if (x,y,) != (x2,y2): 

                # get the nearest network node to each point
                orig_node = ox.get_nearest_node(G, (y,x))
                dest_node = ox.get_nearest_node(G, (y2,x2))

                # how long is our route in meters?
                d = nx.shortest_path_length(G, orig_node, dest_node, weight='length')
                dist_list.append(d)
                lon_list.append(x2)
                lat_list.append(y2)

        df['dist'] = dist_list
        df['lon'] = lon_list
        df['lat'] = lat_list
        site_options[(x,y,)] = df

import math 
def get_dist_haversine(gdf1,gdf2): 
    site_options = {} 
    for x,y in zip(gdf1['lon'],gdf1['lat']):
        df = pd.DataFrame() 
        dist_list = [] 
        lon_list = [] 
        lat_list = [] 
        for x2,y2 in zip(gdf2['lon'],gdf2['lat']): 
            if (x,y,) != (x2,y2): 

                lon1, lat1, lon2, lat2 = map(math.radians, [x, y, x2, y2])

                # haversine formula 
                dlon = lon2 - lon1 
                dlat = lat2 - lat1 
                a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
                c = 2 * math.asin(math.sqrt(a)) 
                r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
                dist = c * r
                dist_list.append(dist)
                lon_list.append(x2)
                lat_list.append(y2)

        df['dist'] = dist_list
        df['lon'] = lon_list
        df['lat'] = lat_list
        site_options[(x,y,)] = df

    return site_options

In [22]:
df_all = road_df.dropna(how='any')
print(list(df_all))

['geometry', 'lat', 'lon', 'fire_detected', 'ca7_detected', 'ca6_detected', 'ca5_detected', 'ca4_detected', 'ca3_detected', 'ca2_detected', 'ca1_detected', 'in_boundary2014', 'in_boundary2015', 'in_boundary2016', 'in_boundary2017', 'in_boundary2018', 'in_boundary2019', 'in_boundary2020', 'age', 'Bf', 'Sw', 'Sb', 'harvest_detected', 'road_detected']


In [56]:
df_all = road_df.dropna(how='any')
#df_all.to_csv('example_data.txt',sep=',')
df_all = df_all[df_all['fire_detected'] == 0]
df_all = df_all[df_all['harvest_detected'] == 0]

#df_all = df_all[df_all['road_detected'] == 1]
df_all['combo_species'] = df_all['Bf'] + df_all['Sw'] +df_all['Sb']
df_all = df_all[df_all['combo_species'] >= 30]
df_all = df_all[df_all['age'] >= 40]

#print(df_all)

site1 = df_all[df_all['ca1_detected'] == 1]
site1 = site1[site1['ca2_detected'] == 0]
site1 = site1[site1['ca3_detected'] == 0]
site1 = site1[site1['ca4_detected'] == 0]
site1 = site1[site1['ca5_detected'] == 0]
site1 = site1[site1['ca6_detected'] == 0]
site1 = site1[site1['ca7_detected'] == 0]

site1 = site1[site1['in_boundary2014'] == 0]
site1 = site1[site1['road_detected'] == 1]

print(site1)

site2 =  df_all[df_all['ca2_detected'] == 1]
site2 = site2[site2['ca3_detected'] == 0]
site2 = site2[site2['ca4_detected'] == 0]
site2 = site2[site2['ca5_detected'] == 0]
site2 = site2[site2['ca6_detected'] == 0]
site2 = site2[site2['ca7_detected'] == 0]
site2 = site2[site2['in_boundary2015'] == 0]
site2 = site2[site2['road_detected'] == 1]

print(site2)

site3 =  df_all[df_all['ca3_detected'] == 1]
site3 = site3[site3['ca4_detected'] == 0]
site3 = site3[site3['ca5_detected'] == 0]
site3 = site3[site3['ca6_detected'] == 0]
site3 = site3[site3['ca7_detected'] == 0]
site3 = site3[site3['in_boundary2016'] == 0]
#site3 = site3[site3['road_detected'] == 1]
print(site3)

site4 =  df_all[df_all['ca4_detected'] == 1]
site4 = site4[site4['ca5_detected'] == 0]
site4 = site4[site4['ca6_detected'] == 0]
site4 = site4[site4['ca7_detected'] == 0]
site4 = site4[site4['in_boundary2017'] == 0]
#site4 = site4[site4['road_detected'] == 1]
print(site4)

site5 =  df_all[df_all['ca5_detected'] == 1]
site5 = site5[site5['ca6_detected'] == 0]
site5 = site5[site5['ca7_detected'] == 0]
site5 = site5[site5['in_boundary2018'] == 0]
site5 = site5[site5['road_detected'] == 1]
print(site5)

site6 =  df_all[df_all['ca6_detected'] == 1]
site6 = site6[site6['ca7_detected'] == 0]
site6 = site6[site6['in_boundary2019'] == 0]
#site6 = site6[site6['road_detected'] == 1]
print(site6)

site7 =  df_all[df_all['ca7_detected'] == 1]
site7 = site7[site7['in_boundary2020'] == 0]
site7 = site7[site7['road_detected'] == 1]
print(site7)


                          geometry        lat        lon  fire_detected  \
388996  POINT (-79.73896 46.36654)  46.366541 -79.738956              0   
400585  POINT (-79.47979 46.36785)  46.367855 -79.479794              0   
400586  POINT (-79.47355 46.36675)  46.366749 -79.473551              0   
400589  POINT (-79.45482 46.36343)  46.363429 -79.454823              0   
401741  POINT (-79.47198 46.37116)  46.371157 -79.471982              0   
409823  POINT (-79.47973 46.40533)  46.405326 -79.479734              0   
409824  POINT (-79.47349 46.40422)  46.404221 -79.473486              0   
410979  POINT (-79.47192 46.40863)  46.408628 -79.471916              0   
431812  POINT (-79.17477 46.43994)  46.439937 -79.174769              0   
434122  POINT (-79.17157 46.44874)  46.448740 -79.171570              0   
436433  POINT (-79.16212 46.45642)  46.456417 -79.162123              0   
437586  POINT (-79.17302 46.46307)  46.463070 -79.173019              0   
458373  POINT (-79.16295 

In [57]:
dd1 = get_dist_haversine(site7,site1) #There are the least site 7's so we optimize distance to them
dd2 = get_dist_haversine(site7,site2)
dd3 = get_dist_haversine(site7,site3)
dd4 = get_dist_haversine(site7,site4)
dd5 = get_dist_haversine(site7,site5)
dd6 = get_dist_haversine(site7,site6)

In [71]:

def optimize_for_site(site,dict1,dict2,dict3,dict4,dict5,dict6):

    information = pd.DataFrame() 

    pot_site1 = dict1[site].reset_index(drop=True)
    pot_site2 = dict2[site].reset_index(drop=True)
    pot_site3 = dict3[site].reset_index(drop=True)
    pot_site4 = dict4[site].reset_index(drop=True)
    pot_site5 = dict5[site].reset_index(drop=True)
    pot_site6 = dict6[site].reset_index(drop=True)

    solver = pywraplp.Solver.CreateSolver('SCIP')

    var_list = []
    dist_list = [] 
    lon_list = [] 
    lat_list = [] 
    var_list_s1 = [] 
    var_list_s2 = []
    var_list_s3 = []
    var_list_s4 = []
    var_list_s5 = []
    var_list_s6 = []

    count = 0 
    for list_ in [pot_site1,pot_site2,pot_site3,\
                  pot_site4,pot_site5,pot_site6]:
        count+=1 
        count2 = 0 #count 2 is the site index 
        for inst in list_.iterrows(): 
            inst = list(inst)[1]
            var_list.append('x'+str(count)+'_'+str(count2))
            dist_list.append(inst['dist'])
            lon_list.append(inst['lon'])
            lat_list.append(inst['lat'])
            if count == 1: 
                var_list_s1.append('x'+str(count)+'_'+str(count2))
            if count == 2: 
                var_list_s2.append('x'+str(count)+'_'+str(count2))
            if count == 3: 
                var_list_s3.append('x'+str(count)+'_'+str(count2))
            if count == 4: 
                var_list_s4.append('x'+str(count)+'_'+str(count2))           
            if count == 5: 
                var_list_s5.append('x'+str(count)+'_'+str(count2))
            if count == 6: 
                var_list_s6.append('x'+str(count)+'_'+str(count2))
            count2+=1 

    var_collect = {}

    for i in range(0, len(var_list)): 
        # solver.IntVar(0.0, 1, var_list[i]) --> tell computer that the variable can
        # only be 1 or 0. 
        var_collect[var_list[i]] = solver.IntVar(0.0, 1, var_list[i]) 
                
    information['variable'] = var_list
    information['distance'] = dist_list 
    information['lat'] = lat_list
    information['lon'] = lon_list

    locals().update(var_collect)
    globals().update(var_collect)
    vars = [eval(i) for i in var_list]
    vars1 = [eval(i) for i in var_list_s1]
    vars2 = [eval(i) for i in var_list_s2]
    vars3 = [eval(i) for i in var_list_s3]
    vars4 = [eval(i) for i in var_list_s4]
    vars5 = [eval(i) for i in var_list_s5]
    vars6 = [eval(i) for i in var_list_s6]

    solver.Add(sum(vars1) == 1)
    solver.Add(sum(vars2) == 1)
    solver.Add(sum(vars3) == 1)
    solver.Add(sum(vars4) == 1)
    solver.Add(sum(vars5) == 1)
    solver.Add(sum(vars6) == 1)

    #print('Number of constraints =', solver.NumConstraints()) 
    #print('Number of variables =', solver.NumVariables())

    #Tell computer that each component of the objective function is dist * variable
    obj_collect = [i*j for i, j in zip(vars,dist_list)]
    #Tell the computer to maximize the sum of these volume*variable pairs 
    solver.Minimize(sum(obj_collect))

    #Solve 
    status = solver.Solve()

    #Get information from solver 
    decision = [] 
    if status == pywraplp.Solver.OPTIMAL:
        
        #print('Objective value =', solver.Objective().Value())
        for var in vars: 
          #Append to decision list the solution for the specific variable. 
          decision.append(abs(var.solution_value()))
    else:
        print('The problem does not have an optimal solution.')

    if len(decision) > 0: 
      #Append a row to the original table IF there is an optimal solution
      information['visit'] = decision 

      #Here's our table. 'cut' tells us whether to harvest or not. 
      #print(information[information['visit'] == 1])
    information = information[information['visit'] == 1]

    
    return float(solver.Objective().Value()),information


In [72]:
op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows(): 
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
    op_list.append(info)
    mdist_list.append(obj_func)
    site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
   variable    distance        lat        lon  visit
18    x1_18   59.342705  46.611759 -79.165535    1.0
19     x2_0   53.331506  46.505601 -79.490534    1.0
20     x3_0  205.159504  47.728553 -82.239948    1.0
22     x4_1  270.945726  48.458619 -82.612107    1.0
33     x5_2   28.391338  47.196570 -79.726342    1.0
36     x6_0  273.402851  48.467510 -82.643787    1.0
Selected Site 7 Location
(-79.77599061218581, 46.94348971582551)
Minimum Haversine Dist Possible Bt Sites (km)
890.5736296261439


In [77]:
#Get another option by removing those sites

op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows():  
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    if loc != (-79.77599061218581, 46.94348971582551,): 
        obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
        op_list.append(info)
        mdist_list.append(obj_func)
        site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
   variable    distance        lat        lon  visit
0      x1_0  122.185721  46.366541 -79.738956    1.0
19     x2_0  138.767617  46.505601 -79.490534    1.0
20     x3_0  144.984781  47.728553 -82.239948    1.0
22     x4_1  229.747733  48.458619 -82.612107    1.0
32     x5_1  127.849366  46.567250 -79.628420    1.0
36     x6_0  231.662560  48.467510 -82.643787    1.0
Selected Site 7 Location
(-81.30084603311286, 46.59170293593686)
Minimum Haversine Dist Possible Bt Sites (km)
995.1977775795654


In [79]:
#And another 

#Get another option by removing those sites

op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows():  
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    if loc != (-79.77599061218581, 46.94348971582551,) and loc != (-81.30084603311286, 46.59170293593686): 
        obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
        op_list.append(info)
        mdist_list.append(obj_func)
        site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
   variable    distance        lat        lon  visit
0      x1_0  120.739409  46.366541 -79.738956    1.0
19     x2_0  137.551374  46.505601 -79.490534    1.0
20     x3_0  147.111127  47.728553 -82.239948    1.0
22     x4_1  231.861046  48.458619 -82.612107    1.0
32     x5_1  126.717765  46.567250 -79.628420    1.0
36     x6_0  233.777674  48.467510 -82.643787    1.0
Selected Site 7 Location
(-81.28612633163375, 46.57544665452763)
Minimum Haversine Dist Possible Bt Sites (km)
997.7583938575408


In [80]:
#And another...

#Get another option by removing those sites

op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows():  
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    if loc != (-79.77599061218581, 46.94348971582551,) and loc != (-81.30084603311286, 46.59170293593686)\
    and loc != (-81.28612633163375, 46.57544665452763): 
        obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
        op_list.append(info)
        mdist_list.append(obj_func)
        site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
   variable    distance        lat        lon  visit
0      x1_0  122.716564  46.366541 -79.738956    1.0
19     x2_0  139.499110  46.505601 -79.490534    1.0
20     x3_0  145.799338  47.728553 -82.239948    1.0
22     x4_1  230.654939  48.458619 -82.612107    1.0
32     x5_1  128.646551  46.567250 -79.628420    1.0
36     x6_0  232.560329  48.467510 -82.643787    1.0
Selected Site 7 Location
(-81.31137149732812, 46.57938773571634)
Minimum Haversine Dist Possible Bt Sites (km)
999.8768296445073


In [81]:
#And another...

#Get another option by removing those sites

op_list = [] 
mdist_list = [] 
site7_loc = [] 
for site in site7.iterrows():  
    site = list(site)[1]
    loc = (site['lon'],site['lat'],)
    if loc != (-79.77599061218581, 46.94348971582551,) and loc != (-81.30084603311286, 46.59170293593686)\
    and loc != (-81.28612633163375, 46.57544665452763) and loc != (-81.31137149732812, 46.57938773571634): 
        obj_func, info = optimize_for_site(loc,dd1,dd2,dd3,dd4,dd5,dd6)
        op_list.append(info)
        mdist_list.append(obj_func)
        site7_loc.append(loc)

#get min

index_min = mdist_list.index(min(mdist_list))

print('Selected Sites')
print(op_list[index_min])
print('Selected Site 7 Location')
print(site7_loc[index_min])
print('Minimum Haversine Dist Possible Bt Sites (km)')
print(mdist_list[index_min])

df_sites = op_list[index_min]

Selected Sites
   variable    distance        lat        lon  visit
0      x1_0  123.721238  46.366541 -79.738956    1.0
19     x2_0  140.557416  46.505601 -79.490534    1.0
20     x3_0  145.533549  47.728553 -82.239948    1.0
22     x4_1  230.460873  48.458619 -82.612107    1.0
32     x5_1  129.718981  46.567250 -79.628420    1.0
36     x6_0  232.358366  48.467510 -82.643787    1.0
Selected Site 7 Location
(-81.3253980335801, 46.576922083322124)
Minimum Haversine Dist Possible Bt Sites (km)
1002.3504229334246


In [78]:
df_sites = df_sites.append(pd.DataFrame([['x0', 0,site7_loc[index_min][1],\
                               site7_loc[index_min][0],1.0]],columns=df_sites.columns))


df_sites['years_act'] = [1,2,3,4,5,6,7]

print(df_sites)
gdf_sites = gpd.GeoDataFrame(df_sites,geometry=\
                             gpd.points_from_xy(df_sites['lon'],df_sites['lat']))
gdf_sites.to_file(driver = 'ESRI Shapefile',filename='op_sites_mort5')
gdf_sites.to_file(driver='GeoJSON',filename='op_sites_mort5')

   variable    distance        lat        lon  visit  years_act
0      x1_0  122.185721  46.366541 -79.738956    1.0          1
19     x2_0  138.767617  46.505601 -79.490534    1.0          2
20     x3_0  144.984781  47.728553 -82.239948    1.0          3
22     x4_1  229.747733  48.458619 -82.612107    1.0          4
32     x5_1  127.849366  46.567250 -79.628420    1.0          5
36     x6_0  231.662560  48.467510 -82.643787    1.0          6
0        x0    0.000000  46.591703 -81.300846    1.0          7


IsADirectoryError: ignored